In [ ]:
#Load python packages and set some defaults

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from landlab import RasterModelGrid 
from landlab.components import TruckPassErosion
from landlab.components import KinwaveImplicitOverlandFlow
from landlab.plot.imshow import imshow_grid

mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.weight'] = 'medium'
mpl.rcParams['axes.labelweight'] = 'normal'

The first thing we're going to do is create a synthetic grid (i.e., make our own DEM!) with a ditch line. We're going to create it as a `python` function so we can easily change node spacing depending on whether we're using a node size of a full-tire width or a half-tire-width.

In [ ]:
# Create erodible grid function
def ErodibleGrid(nrows, ncols, spacing, full_tire):
    mg = RasterModelGrid((nrows,ncols),spacing)
    z = mg.add_zeros('topographic__elevation', at='node') #create the topographic__elevation field
    road_flag = mg.add_zeros('flag', at='node') #create a road_flag field for determining whether a 
                                                #node is part of the road or the ditch line

    mg.set_closed_boundaries_at_grid_edges(False, False, False, False) 
    
    if full_tire == False:
        road_peak = 40 #peak crowning height occurs at this x-location
        up = 0.0067 #rise of slope from ditchline to crown
        down = 0.0067 #rise of slope from crown to fillslope
        
        for g in range(nrows): #loop through road length
            elev = 0 #initialize elevation placeholder
            flag = False #initialize road_flag placeholder

            for h in range(ncols): #loop through road width
                z[g*ncols + h] = elev #update elevation based on x & y locations
                road_flag[g*ncols+h] = flag #update road_flag based on x & y locations

                if h == 0 or h == 8:
                    elev = 0
                    flag = False
                elif h == 1 or h == 7:
                    elev = -0.333375
                    flag = False
                elif h == 2 or h == 6:
                    elev = -0.5715
                    flag = False
                elif h == 3 or h == 5:
                    elev = -0.714375
                    flag = False
                elif h == 4:
                    elev = -0.762
                    flag = False
                elif h < road_peak and h > 7: #update latitudinal slopes based on location related to road_peak
                    elev += up
                    flag = True
                else:
                    elev -= down
                    flag = True
    elif full_tire == True:
        road_peak = 20 #peak crowning height occurs at this x-location
        up = 0.0134 #rise of slope from ditchline to crown
        down = 0.0134 #rise of slope from crown to fillslope
        
        for g in range(nrows): #loop through road length
            elev = 0 #initialize elevation placeholder
            flag = False #initialize road_flag placeholder

            for h in range(ncols): #loop through road width
                z[g*ncols + h] = elev #update elevation based on x & y locations
                road_flag[g*ncols+h] = flag #update road_flag based on x & y locations

                if h == 0 or h == 4:
                    elev = 0
                    flag = False
                elif h == 1 or h == 3:
                    elev = -0.5715
                    flag = False
                elif h == 2:
                    elev = -0.762
                    flag = False
                elif h < road_peak and h > 3: #update latitudinal slopes based on location related to road_peak
                    elev += up
                    flag = True
                else:
                    elev -= down
                    flag = True
        
    z += mg.node_y*0.05 #add longitudinal slope to road segment
    road_flag = road_flag.astype(bool) #Make sure road_flag is a boolean array

    n = mg.add_zeros('roughness', at='node') #create roughness field
    
    roughness = 0.1 #initialize roughness placeholder            
    
    for g in range(nrows): #loop through road length
        for h in range(ncols): #loop through road width
            n[g*ncols + h] = roughness #update roughness values based on x & y locations
            
            if h >= 0 and h <= 8: #ditchline Manning's n value is higher than OF
                roughness = 0.1
            else:
                roughness = 0.02
                
    return(mg, z, road_flag, n)  